In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# Load the data
data = pd.read_excel("questionnaire_answer_107.xlsx")
reliability_data = data.iloc[:, 1:]

def compute_cronbach_alpha(data):
    """
    Compute Cronbach's Alpha for a dataset.
    """
    num_items = data.shape[1]
    item_variances = data.var(axis=0, ddof=1)
    total_variances = data.sum(axis=1).var(ddof=1)
    return num_items / (num_items - 1.) * (1 - item_variances.sum() / total_variances)

def split_half_reliability(data):
    """
    Compute the split-half reliability for a dataset.
    This function computes the average split-half reliability over multiple random splits.
    """
    num_splits = 500  # Reduced for performance
    reliabilities = []
    num_items = data.shape[1]
    for _ in range(num_splits):
        permutation = np.random.permutation(num_items)
        half_1 = data.iloc[:, permutation[:num_items // 2]]
        half_2 = data.iloc[:, permutation[num_items // 2:]]
        total_1 = half_1.sum(axis=1)
        total_2 = half_2.sum(axis=1)
        r, _ = pearsonr(total_1, total_2)
        reliability = 2 * r / (1 + r)
        reliabilities.append(reliability)
    return np.mean(reliabilities)

def compute_mcdonald_omega(data):
    """
    Compute McDonald's Omega for a dataset.
    Currently, this is an approximation using Cronbach's Alpha.
    """
    return compute_cronbach_alpha(data)

def bootstrap_confidence_interval(data, stat_function, alpha=0.05, n_iterations=500):
    """
    Compute the confidence interval of a statistic using bootstrap.
    """
    estimates = []
    for _ in range(n_iterations):
        sample = data.sample(n=data.shape[0], replace=True, axis=0)
        estimates.append(stat_function(sample))
    lower = np.percentile(estimates, 100 * (alpha / 2.))
    upper = np.percentile(estimates, 100 * (1 - alpha / 2.))
    return lower, upper

# Compute the statistics and 95% CIs
cronbach_alpha_mean = compute_cronbach_alpha(reliability_data)
cronbach_alpha_lower, cronbach_alpha_upper = bootstrap_confidence_interval(reliability_data, compute_cronbach_alpha)

split_half_mean = split_half_reliability(reliability_data)
split_half_lower, split_half_upper = bootstrap_confidence_interval(reliability_data, split_half_reliability)

# Using bootstrap for McDonald's ω
mcdonald_omega_mean = compute_mcdonald_omega(reliability_data)
mcdonald_omega_lower, mcdonald_omega_upper = bootstrap_confidence_interval(reliability_data, compute_mcdonald_omega)

print('next')
# # Construct the table
# results = {
#     'Estimate': ['Posterior mean', '95% CI lower bound', '95% CI upper bound'],
#     'McDonald’s ω': [mcdonald_omega_mean, mcdonald_omega_lower, mcdonald_omega_upper],
#     'Cronbach’s α': [cronbach_alpha_mean, cronbach_alpha_lower, cronbach_alpha_upper],
#     'Guttman’s λ2': [split_half_mean, split_half_lower, split_half_upper]
# }

# table = pd.DataFrame(results)
# print(table)


next


In [2]:
import pandas as pd
from reliabilipy import reliability_analysis

# Load the data
data = pd.read_excel("questionnaire_answer_107.xlsx")
reliability_data = data.iloc[:, 1:]

# Compute the correlation matrix of the data
correlations_matrix = reliability_data.corr()

# Create the reliability report using reliabilipy
reliability_report = reliability_analysis(correlations_matrix=correlations_matrix)
reliability_report.fit()

# Extract the reliability measures
omega_total = reliability_report.omega_total
omega_hierarchical = reliability_report.omega_hierarchical
omega_hierarchical_asymptotic = reliability_report.omega_hierarchical_asymptotic
alpha_cronbach = reliability_report.alpha_cronbach
lambda1 = reliability_report.lambda1
lambda2 = reliability_report.lambda2

# Print the results
print(f"Omega Total: {omega_total}")
print(f"Omega Hierarchical: {omega_hierarchical}")
print(f"Omega Hierarchical Asymptotic: {omega_hierarchical_asymptotic}")
print(f"Cronbach’s Alpha: {alpha_cronbach}")
print(f"Guttman's Lambda 1: {lambda1}")
print(f"Guttman's Lambda 2: {lambda2}")

Omega Total: 0.9824127237685539
Omega Hierarchical: 0.2585516918443122
Omega Hierarchical Asymptotic: 0.26318031677409776
Cronbach’s Alpha: 0.9694273027610495
Guttman's Lambda 1: 0.9603672345109461
Guttman's Lambda 2: 0.9759194641010885


In [3]:
# Construct the table
results_2 = {
    'Estimate': ['Posterior mean', '95% CI lower bound', '95% CI upper bound'],
    'McDonald’s ω': [omega_total, mcdonald_omega_lower, mcdonald_omega_upper],
    'Cronbach’s α': [alpha_cronbach, cronbach_alpha_lower, cronbach_alpha_upper],
    'Guttman’s λ2': [lambda2, split_half_lower, split_half_upper]
}

table = pd.DataFrame(results_2)
print(table)

             Estimate  McDonald’s ω  Cronbach’s α  Guttman’s λ2
0      Posterior mean      0.982413      0.969427      0.975919
1  95% CI lower bound      0.945810      0.950115      0.953569
2  95% CI upper bound      0.981701      0.981059      0.986213
